In [1]:
import json
import altair as alt
import pandas as pd
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [2]:
data_files = {
    'No nulls': 'results_0.json',
    '10% nulls': 'results_10.json',
    '50% nulls': 'results_50.json'
}

all_results = []

for case, filename in data_files.items():
    case_results = json.loads(open(filename).read())['benchmarks']
    for result in case_results:
        result['case'] = case
        all_results.append(result)

results = pd.DataFrame.from_records(all_results)

In [3]:
results

,case,cpu_time,iterations,name,real_time,time_unit
0,No nulls,12309.404017,59,BM_SumDoubleNoNulls,12309.008645,us
1,No nulls,5417.733603,126,BM_SumDoubleNoNullsBatched,5417.676619,us
2,No nulls,12055.371169,59,BM_SumDoubleWithNaN,12055.110914,us
3,No nulls,12419.343421,57,BM_SumDoubleWithNaNVectorize,12419.837492,us
4,No nulls,12117.465828,58,BM_SumDoubleBitmapNaive,12117.227465,us
5,No nulls,12353.718218,55,BM_SumDoubleBitmapReader,12353.256436,us
6,No nulls,5592.701692,120,BM_SumDoubleBitmapVectorize,5593.000349,us
7,10% nulls,12089.845754,57,BM_SumDoubleNoNulls,12090.847107,us
8,10% nulls,4918.600042,142,BM_SumDoubleNoNullsBatched,4918.423485,us
9,10% nulls,14903.660348,46,BM_SumDoubleWithNaN,14904.965371,us


In [7]:
to_plot = results.copy()
to_plot.name = to_plot.name.map(
  {'BM_SumDoubleBitmapNaive': 'Naive bitmap',
   'BM_SumDoubleNoNulls': 'Naive no nulls',
   'BM_SumDoubleWithNaN': 'Naive NaN',
   'BM_SumDoubleBitmapVectorize': 'Vectorized bitmap',
   'BM_SumDoubleNoNullsBatched': 'Vectorized no nulls',
   'BM_SumDoubleWithNaNVectorize': 'Vectorized NaN'
   }
)

to_plot.case = pd.Categorical(to_plot.case, 
                              categories=['No nulls', '10% nulls', '50% nulls'])
to_plot = to_plot[to_plot.name.notnull()]

chart = alt.Chart(to_plot).mark_bar().encode(
  x=alt.X('cpu_time'),
  row='case:N',
  y=alt.Y('name:N')
)

chart.width = 500
chart.title = 'Time to sum 10M values (in microseconds)'
chart.save('chart.html', format='html')

In [40]:
pd.Categorical?